## Digit Recognition Project

In [1]:
import numpy as np
import pandas as pd

from data_utils import plot_images_with_labels, get_augmented_data
from grid_search import get_best_pipeline, train_validate_test_split

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

import copy

import matplotlib.pyplot as plt

### A. Data Exploration and Preprocessing
1. Load and inspect the dataset: Discuss the meaning of the data and make sure that they are properly loaded. For example, are all data values sensible? Please include a figure or table to illustrate that the data is properly loaded.
2. Perform the necessary steps to put the data in the right format for the machine learning algorithms. Explain the steps you take and why you take them. You are expected to  use all features and images – there is no need for outlier pruning or other fancy preprocessing techniques.


#### Load and Inspect the Dataset

In [ ]:
# Load the dataset and plot some samples

labeled_images = np.load("labeled_images.npy")
labeled_digits = np.load("labeled_digits.npy")

autograder_images = np.load("autograder_images.npy")

dataset = (labeled_images, labeled_digits)

plot_images_with_labels(dataset, num_images=16)

In [ ]:
# Inspect the dataset

# Inspect shapes of the datasets
dataset_shape = labeled_images.shape
labels_shape = labeled_digits.shape

print(f"Original dataset shapes: {labeled_images.shape}, {labeled_digits.shape}")

# Compute basic statistics for labeled_images
images_min = np.min(labeled_images)
images_max = np.max(labeled_images)
images_mean = np.mean(labeled_images)
images_std = np.std(labeled_images)

labels_min = np.min(labeled_digits)
labels_max = np.max(labeled_digits)
labels_mean = np.mean(labeled_digits)
labels_std = np.std(labeled_digits)
labels_median = np.median(labeled_digits)

# Check the unique labels in labeled_digits
unique_labels = np.unique(labeled_digits)

# Prepare a summary table
summary_table = pd.DataFrame({
    "Attribute": ["Shape", "Min", "Max", "Mean", "Standard Deviation", "Median", "Unique Labels"],
    "Images": [str(dataset_shape), images_min, images_max, images_mean, images_std, None, None],
    "Labels": [str(labels_shape), labels_min, labels_max, labels_mean, labels_std, labels_median, unique_labels.tolist()]
})

# Save the label data summary as a markdown file
markdown_content = summary_table.to_markdown(index=False)

# Save to a .md file
file_path = "report/data_summary.md"
with open(file_path, "w") as f:
    f.write(markdown_content)

summary_table

In [ ]:
# Compute the distribution of labels
label_counts = np.bincount(labeled_digits)

# Create a DataFrame to summarize the distribution
label_distribution = pd.DataFrame({
    "Label": range(len(label_counts)),
    "Count": label_counts,
    "Percentage": (label_counts / len(labeled_digits)) * 100
})

# Save the label distribution as a markdown file
markdown_content = label_distribution.to_markdown(index=False)

# Save to a .md file
file_path = "report/label_distribution.md"
with open(file_path, "w") as f:
    f.write(markdown_content)

label_distribution

#### Data Preprocessing and Augmentation

In [ ]:
# Augment the data and plot some samples
augmented_dataset = get_augmented_data(dataset)
plot_images_with_labels(dataset, num_images=16)

In [ ]:
augmented_images, augmented_labels = augmented_dataset

# Inspect the dataset

# Inspect shapes of the datasets
dataset_shape = augmented_images.shape
labels_shape = augmented_labels.shape

print(f"Augmented dataset shapes: {augmented_images.shape}, {augmented_labels.shape}")

# Compute basic statistics for augmented_images
images_min = np.min(augmented_images)
images_max = np.max(augmented_images)
images_mean = np.mean(augmented_images)
images_std = np.std(augmented_images)

labels_min = np.min(augmented_labels)
labels_max = np.max(augmented_labels)
labels_mean = np.mean(augmented_labels)
labels_std = np.std(augmented_labels)
labels_median = np.median(augmented_labels)

# Check the unique labels in labels
unique_labels = np.unique(augmented_labels)

# Prepare a summary table
summary_table = pd.DataFrame({
    "Attribute": ["Shape", "Min", "Max", "Mean", "Standard Deviation", "Median", "Unique Labels"],
    "Images": [str(dataset_shape), images_min, images_max, images_mean, images_std, None, None],
    "Labels": [str(labels_shape), labels_min, labels_max, labels_mean, labels_std, labels_median, unique_labels.tolist()]
})

# Save the label data summary as a markdown file
markdown_content = summary_table.to_markdown(index=False)

# Save to a .md file
file_path = "report/augmented_data_summary.md"
with open(file_path, "w") as f:
    f.write(markdown_content)

summary_table

In [ ]:
# Compute the distribution of labels
label_counts = np.bincount(augmented_labels)

# Create a DataFrame to summarize the distribution
label_distribution = pd.DataFrame({
    "Label": range(len(label_counts)),
    "Count": label_counts,
    "Percentage": (label_counts / len(augmented_labels)) * 100
})

# Save the label distribution as a markdown file
markdown_content = label_distribution.to_markdown(index=False)

# Save to a .md file
file_path = "report/augmented_label_distribution.md"
with open(file_path, "w") as f:
    f.write(markdown_content)

label_distribution

### B. Regression with Default Hyperparameters
1. What is the simplest baseline model we should aim to beat? Or in other words, if you would have to make a guess for the label without knowing anything about the image, what would you guess? What is the accuracy of such a guess?
2. Train the 4 models with default hyperparameters and fairly estimate their performance. Explain why the performance estimate is fair and how you estimated the performance.
3. Submit your work to the autograder to check your work so far.

### C. Tuning with GridSearch
1. For all 4 models, use GridSearch (or any other strategy) to identify the best hyperparameters. Use a systematic way to tune hyperparameters that is reproducible. 
2. Explain your choice of hyperparameter search ranges and settings. Include sufficient details in the report so that another student can reproduce your experiment.
3. Include a training curve (accuracy versus epochs) to illustrate how SGD converges.
Explain why the performance estimate is fair for the tuned models and how you estimated the performance.
4. Compare the performance of all models before and after hyperparameter tuning


#### Dummy Classifier

```python
class sklearn.dummy.DummyClassifier(*, strategy='prior', random_state=None, constant=None)
```

In [ ]:
dummy = DummyClassifier(strategy="most_frequent")

dummy_results = get_best_pipeline(dataset, dummy, {})

# Output test set accuracy
print(f"Test set accuracy: {dummy_results[2]:.4f}")

In [ ]:
aug_dummy = DummyClassifier(strategy="most_frequent")

aug_dummy_results = get_best_pipeline(dataset, aug_dummy, {}, data_aug=True)

# Output test set accuracy
print(f"Test set accuracy: {aug_dummy_results[2]:.4f}")

#### Stochastic Gradient Descent Classifier with Logistic Loss Function

```python
class sklearn.linear_model.SGDClassifier(loss='hinge', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
```

##### Default Hyperparamters

In [ ]:
# Without data augmentation

default_SGD = SGDClassifier(loss="log_loss")

default_SGD_results = get_best_pipeline(dataset, default_SGD, {})

# Output test set accuracy
print(f"Test set accuracy: {default_SGD_results[2]:.4f}")

In [ ]:
# With data augmentation

aug_default_SGD = SGDClassifier(loss="log_loss")

aug_default_SGD_results = get_best_pipeline(dataset, aug_default_SGD, {}, data_aug=True)

# Output test set accuracy
print(f"Test set accuracy: {aug_default_SGD_results[2]:.4f}")

##### GridSearch Cross-Validation

In [ ]:
# Without data augmentation

gridsearch_SGD = SGDClassifier(
        loss="log_loss"
    )


param_grid = {
    "model__alpha": [0.0001, 0.001, 0.01],
    "model__learning_rate": ["constant", "optimal"],
    "model__eta0": [0.001, 0.01, 0.1]
}

gridsearch_SGD_results = get_best_pipeline(dataset, gridsearch_SGD, param_grid)

print(f"Test set accuracy: {gridsearch_SGD_results[2]:.4f}")

In [ ]:
# With data augmentation
aug_gridsearch_SGD = SGDClassifier(
        loss="log_loss"
    )


param_grid = {
    "model__alpha": [0.0001, 0.001, 0.01],
    "model__learning_rate": ["constant", "optimal"],
    "model__eta0": [0.001, 0.01, 0.1]
}

aug_gridsearch_SGD_results = get_best_pipeline(dataset, aug_gridsearch_SGD, param_grid, data_aug=True)

# Output test set accuracy
print(f"Test set accuracy: {aug_gridsearch_SGD_results[2]:.4f}")

##### Plot the Training Curve

In [14]:
def plot_SGD_training_curve(best_params, dataset, max_epochs=1000, step=25, data_aug=False):
    """
    Trains an SGDClassifier with the best parameters incrementally and plots the training curve (accuracy vs. epochs).

    Args:
        best_params (dict): The best hyperparameters for SGDClassifier.
        dataset (tuple): A tuple containing (images, labels).
        max_epochs (int, optional): Maximum number of epochs to train. Default is 1000.
        step (int, optional): Step size for epoch increments. Default is 25.
        data_aug (bool, optional): Whether to perform data augmentation. Default is False.

    Returns:
        best_model (SGDClassifier): The best-performing model observed during training.
        None: Displays the training curve plot.
    """
    # Unpack dataset
    (X_train_val, y_train_val), (X_test, y_test) = train_validate_test_split(dataset, test_size=0.2)
    
    # Prepare data for training
    if data_aug:
        X_train_val, y_train_val = get_augmented_data((X_train_val, y_train_val))
    X_train_val_flat = X_train_val.reshape(X_train_val.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    classes = np.unique(y_train_val)  # Ensure all classes are passed for partial_fit

    # Initialize the model with the best parameters
    model = SGDClassifier(
        loss="log_loss",
        alpha=best_params["model__alpha"],
        learning_rate=best_params["model__learning_rate"],
        eta0=best_params["model__eta0"],
        max_iter=1,  # Set max_iter to 1 for incremental updates
        warm_start=True,  # Allows continuation of training
        random_state=42,
        tol=None,  # Ensure it doesn't stop early
    )

    # Track accuracies
    train_accuracies = []
    test_accuracies = []
    epoch_range = range(1, max_epochs + 1, step)

    # Variables to track the best model
    best_model = None
    best_test_accuracy = 0.0

    for epoch in epoch_range:
        # Incrementally train the model for `step` epochs
        for _ in range(step):
            model.partial_fit(X_train_val_flat, y_train_val, classes=classes)

        # Compute accuracies
        train_accuracy = model.score(X_train_val_flat, y_train_val)
        test_accuracy = model.score(X_test_flat, y_test)

        train_accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)

        # Check if this model is the best so far
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            best_model = copy.deepcopy(model)  # Save the current best model

    # Plot the training curve
    plt.figure(figsize=(8, 6))
    plt.plot(epoch_range, train_accuracies, label="Training Accuracy", marker="o")
    plt.plot(epoch_range, test_accuracies, label="Test Accuracy", marker="s")
    plt.xlabel("Number of Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training Curve (Incremental Training): Accuracy vs. Epochs")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    print(f"Best Test Accuracy: {best_test_accuracy:.4f}")
    return best_model


In [ ]:
_, best_params, _  = gridsearch_SGD_results
# Call the function with the best parameters and pipeline
best_SGD = plot_SGD_training_curve(best_params, dataset)

In [ ]:
_, best_params, _  = aug_gridsearch_SGD_results
# Call the function with the best parameters and pipeline
best_aug_SGD = plot_SGD_training_curve(best_params, dataset, data_aug=True)

#### Decision Tree

```python
class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0, monotonic_cst=None)
```

##### Default Hyperparameters

In [ ]:
# Without data augmentation

default_DTC = DecisionTreeClassifier()

default_DTC_results = get_best_pipeline(dataset, default_DTC, {})

# Output test set accuracy
print(f"Test set accuracy: {default_DTC_results[2]:.4f}")

In [ ]:
# With data augmentation

aug_default_DTC = DecisionTreeClassifier()

aug_default_DTC_results = get_best_pipeline(dataset, aug_default_DTC, {}, data_aug=True)

# Output test set accuracy
print(f"Test set accuracy: {aug_default_DTC_results[2]:.4f}")

##### GridSearch Cross-Validation

In [ ]:
# Without data augmentation

gridsearch_DTC = DecisionTreeClassifier()


param_grid = {
    "model__max_depth": [5, 10, 20, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 5, 10],
    "model__max_features": [None, "sqrt", "log2"]
}

gridsearch_DTC_results = get_best_pipeline(dataset, gridsearch_DTC, param_grid)

print(f"Test set accuracy: {gridsearch_DTC_results[2]:.4f}")

In [ ]:
# With data augmentation

aug_gridsearch_DTC = DecisionTreeClassifier()


param_grid = {
    "model__max_depth": [5, 10, 20, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 5, 10],
    "model__max_features": [None, "sqrt", "log2"]
}

aug_gridsearch_DTC_results = get_best_pipeline(dataset, aug_gridsearch_DTC, param_grid, data_aug=True)

print(f"Test set accuracy: {aug_gridsearch_DTC_results[2]:.4f}")

#### SVM Classifier

```python 
class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)
```

##### Default Hyperparameters

In [ ]:
# Without data augmentation

default_SVC = SVC(
        C=1.0, kernel='rbf', gamma='scale' 
    )

default_SVC_results = get_best_pipeline(dataset, default_SVC, {})

# Output test set accuracy
print(f"Test set accuracy: {default_SVC_results[2]:.4f}")

In [ ]:
# With data augmentation

aug_default_SVC = SVC(
        C=1.0, kernel='rbf', gamma='scale' 
    )

aug_default_SVC_results = get_best_pipeline(dataset, aug_default_SVC, {}, data_aug=True)

# Output test set accuracy
print(f"Test set accuracy: {aug_default_SVC_results[2]:.4f}")

##### GridSearch Cross Validation 

In [ ]:
# Without data augmentation

gridsearch_SVC = SVC(
        C=1, kernel='rbf', gamma='scale'
    )


param_grid = {
    "model__C": [1, 2, 3, 4, 5, 6, 7, 8, 9], 
    "model__gamma": [0.001, 0.1, 1, 10],
    "model__kernel": ['rbf', 'poly', 'sigmoid'],
    "model__degree": [2, 4, 8, 10]
}

gridsearch_SVC_results = get_best_pipeline(dataset, gridsearch_SVC, param_grid)

print(f"Test set accuracy: {gridsearch_SVC_results[2]:.4f}")

In [ ]:
# With data augmentation

aug_gridsearch_SVC = SVC(
        C=1, kernel='rbf', gamma='scale'
    )


param_grid = {
    "model__C": [1, 2, 3, 4, 5, 6, 7, 8, 9], 
    "model__gamma": [0.001, 0.1, 1, 10],
    "model__kernel": ['rbf', 'poly', 'sigmoid'],
    "model__degree": [2, 4, 8, 10]
}


aug_gridsearch_SVC_results = get_best_pipeline(dataset, aug_gridsearch_SVC, param_grid, data_aug=True)

print(f"Test set accuracy: {aug_gridsearch_SVC_results[2]:.4f}")

### (SVC) Prepare autograder submission

In [19]:

estimate = np.array(aug_gridsearch_SVC_results[2])
prediction = aug_gridsearch_SVC_results[0].predict(autograder_images)
# For example using something like:
# prediction = my_super_duper_model.predict(autograder_images) 

result = np.append(estimate, prediction)

# The code below will write your estimate and prediction to a file named autograder.txt
# You will need to upload this file to the Vocareum autograder
pd.DataFrame(result).to_csv("autograder_svc.txt", index=False, header=False)

### Prepare autograder submission

In the autograder you will need to provide two things: 1) estimate of the accuracy of your model on unseen data, 2) the predictions on the autograder images. For the autograder images we only provide the images and not the class labels. Thus, you cannot compute the accuracy on this data yourself - you need to estimate that with labeled data that is provided (labeled_images, labeled_digits). We will calculate the accuracy for you on the autograder data and you will receive an automatic grade based on this. 

In [ ]:

estimate = np.array([0.7]) # TODO Replace this with your estimate of the accuracy on new data
prediction = np.array([-1] * len(autograder_images)) # TODO Replace this with your predictions of your best model
# For example using something like:
# prediction = my_super_duper_model.predict(autograder_images) 

result = np.append(estimate, prediction)

# The code below will write your estimate and prediction to a file named autograder.txt
# You will need to upload this file to the Vocareum autograder
pd.DataFrame(result).to_csv("autograder.txt", index=False, header=False)